In [1]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

openai_llm = OpenAI(temperature=0.4)

### Example 1: Using PromptTemplate -- write a list of jokes, then a second list of jokes very similar to the first one but on a different topic

In [8]:
professions = ['computer scientists', 'truck drivers', 'lawyers', 'accountants']

In [4]:
profession1 = professions[0]
prompt1 = f"Write 7 jokes about {profession1}."
resp1 = openai_llm(prompt1)

In [5]:
print (resp1.strip())

1. What did the computer scientist say when he found a big bug in his code? "That's not a bug, that's a feature!"

2. Why did the computer scientist cross the road? To get to the other side of the algorithm.

3. What did the computer scientist say when he was asked to debug a program? "I'm no magician, I'm a computer scientist!"

4. What did the computer scientist say when his computer crashed? "It must have been a logic error!"

5. Why did the computer scientist keep dropping his keys? He was trying to find the key to the algorithm.

6. How did the computer scientist propose to his girlfriend? With a binary search tree.

7. What did the computer scientist say when he saw a broken computer? "It looks like it needs a few lines of code."


In [9]:
template = """
Given below is a numbered list of jokes about {profession1}. 
Riff on these jokes to make equal number of very similar jokes about {profession2}. 
Do not add any explanation, elaboration or any additional comments.
---
{jokes_output}
"""
p_template = PromptTemplate(input_variables=['profession1', 'profession2', 'jokes_output'], template=template)

In [10]:
for profession2 in professions[1:]:
    prompt2 = p_template.format(profession2=profession2, jokes_output=resp1, profession1=profession1)
    resp2 = openai_llm(prompt2)
    print (f"======================={profession2}=====================")
    print (resp2.strip())

=======================truck drivers=====================
1. What did the truck driver say when he found a big pothole in the road? "That's not a pothole, that's a shortcut!"

2. Why did the truck driver cross the road? To get to the other side of the highway.

3. What did the truck driver say when he was asked to drive through a storm? "I'm no miracle worker, I'm a truck driver!"

4. What did the truck driver say when his truck broke down? "It must have been a fuel error!"

5. Why did the truck driver keep dropping his keys? He was trying to find the key to the ignition.

6. How did the truck driver propose to his girlfriend? With a long haul.

7. What did the truck driver say when he saw a broken-down truck? "It looks like it needs a few gallons of gas."
=======================lawyers=====================
1. What did the lawyer say when he found a loophole in the contract? "That's not a loophole, that's a feature!"

2. Why did the lawyer cross the road? To get to the other side of th

### Example 2 : using LLMchain